In [1]:
import pandas as pd
from pandas import Series, DataFrame
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Q1 [12 points] Classification

Consider the following 2 datasets, each of which has equal numbers of red and blue points.

![both](both.png)

### Q1(a) [4 points] Suppose you can build a decision tree to classify the red points from the blue points. 
You can set the depth anywhere between 1 and 10 (recall that depth 1 means you get to split at most once).
Which depth gives you the best training accuracy for dataset 1? And for dataset 2?
If several choices for depth achieve the highest accuracy, report the smallest depth that achieves that accuracy.

**Fill in**

### Q1(b) [4 points] What is the accuracy achieved by logistic regression on dataset 1? On dataset 2?

For dataset 2, you may give an approximate answer assuming that the red and blue circles are very very close to each other.

**Fill in**

### Q1(c) [4 points] What accuracy would a K-Nearest Neighbors algorithm achieve on dataset 1? On dataset 2?

You may assume a reasonably large value for K, say, K=20. You can also assume that the points are uniformly spread out over the circles, and for dataset 2, the two circles are very very close to each other.

**Fill in**

---

We will consider a dataset where each row is one active businesses in the Los Angeles area. We care about the following fields:
* BUSINESS NAME, STREET ADDRESS, CITY, ZIP CODE: self-explanatory
* MAILING ADDRESS, MAILING CITY, MAILING ZIP CODE: these correspond to locations where letters addressed to the business need to be mailed
* LOCATION DESCRIPTION: a column that combines the street name and zip code
* NAICS: North American Industry Classification System (essentially the type of business)
* PRIMARY NAICS DESCRIPTION: a description of the NAICS code
* COUNCIL DISTRICT: another location identifier
* LOCATION START DATE: when did the business start using this location.

*Note*: You can assume that STREET ADDRESS has no NaNs. However, **there may be NaNs in all other columns, and your code may need to account for that if necessary!**

### Q2 [3 points] Read in "ActiveBusinesses.csv" and print the first line.

### Q3 [4 points] Businesses with the same street address.

Find the most frequent STREET ADDRESS (i.e., the STREET ADDRESS that shows up in the most rows). List all the BUSINESS NAMEs at that STREET ADDRESS.

### Q4 [4 points] Plot the top 10 most frequent BUSINESS NAMEs.
The x-axis should be the BUSINESS NAME, and the y-axis should be the number of rows with that BUSINESS NAME.

### Q5 [4 points] Top street names

Define the street name as the LOCATION DESCRIPTION without the first word (street number) and the last word (the zip code).
Print the frequencies of the top 10 street names.

### Q6 [4 points] Plot the frequencies of the top-10 cities except LOS ANGELES.

### Q7 [4 points] Among rows where both ZIP CODE and MAILING ZIP CODE are not missing, what fraction of rows have different values for these two columns?

### Q8 [4 points] How many unique STREET ADDRESSes have a single character as their second word?

For example, the second word of "1155 N LA CIENEGA BLVD" is "N".

### Q9 [4 points] Among rows where NAICS is not missing, what fraction also have PRIMARY NAICS DESCRIPTION also not missing?

### Q10 [4 points] Find all unique BUSINESS NAMEs where the CITY in which the business is located is part of the BUSINESS NAME.

For example, "LOS ANGELES VALLEY CAREGIVERS, INC." in the city of LOS ANGELES.

### Q11 [4 points] Print out all cities with 5 or more unique council districts associated with them.

### Q12 [6 points] Multiple NAICS
Find all instances where there are several different NAICS for the same PRIMARY NAICS DESCRIPTION. Print out these NAICS and PRIMARY NAICS DESCRIPTIONs.

### Q13 [6 points] Plot the frequency of street last-names.

The street last-name is defined as follows:
* Look at a STREET ADDRESS (e.g., "5670 WILSHIRE BLVD #B 1")
* Read its words backwards (here:, "1", "#B", "BLVD", "WILSHIRE", "5670")
* Pick the first word that doesn't have any digits or "#" in it (here: "BLVD")

Calculate the number of rows associated with each unique last-name. Plot the frequencies of the top 10 last-names.

### Q14 [6 points] Streets with the widest range of street numbers

* Drop all STREET ADDRESSES where the first word is not all numbers. For example, keep the row where the STREET ADDRESS is "15136 STAGG STREET" but drop the row with "POST OFFICE BOX 123".
* Among these STREET ADDRESSES, call the first word STREET NUMBER and the rest of the address STREET BASE.
    * For example, a STREET ADDRESS of "15136 STAGG STREET" has a STREET NUMBER of 15136 and a STREET BASE of "STAGG STREET"
* For each STREET BASE, figure out the range of STREET NUMBERs.
    * For example, if you see "15126 STAGG STREET", "15100 STAGG STREET" and "15200 STAGG STREET", then the range for "STAGG STREET" is 15200-15100=100.
* Print out the top 5 STREET BASEs with the largest range of STREET NUMBERs.

### Q15 [6 points] Among MAILING CITIES that appear at least 50 times, what is the fraction of rows for which the MAILING ADDRESS contains the string "POST OFFICE BOX"?

### Q16 [6 points] Business chains

* Find all BUSINESS NAMEs that are associated with 6 or more unique non-missing LOCATIONs.
* For each such BUSINESS NAME, print out a DataFrame with the index being the BUSINESS NAME and two columns:
    * the number of unique cities in which the business operates, and
    * the CITY with the maximum number of locations for this business (you need not consider unique locations here).

### Q17 [6 points] Visualize Ventura

* Find all rows where the STREET ADDRESS contains "VENTURA BLVD".
* Look at the LOCATION column for these rows. These are in the form (latitude, longitude)
* Drop any rows where the LOCATION is missing or equals "(0.0, 0.0)"
* Extract the latitudes and longitudes. 
* Show a scatter plot of locations with latitude on the y-axis and longitude on the x-axis. These give a visualization of roads named Ventura Blvd.

### Q18 [4 points] Plot the total number of rows for each month from 01/01/2000 to 01/01/2021.

---

### Autism Classification

We will consider a dataset where each row is an adult who may be autistic (Class/ASD is YES). We will only need the columns named A1_Score to A10_Score. These are scores achieved on answers to different questions measuring mental acuity. A score of 1 is a correct answer, while 0 is a wrong answer.

In [2]:
df = pd.read_csv('Autism-Adult-Data.arff', skiprows=25, header=None, \
                 names=['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
                        'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age',
                        'gender', 'ethnicity', 'jaundice', 'austim', 'country_of_res',
                        'used_app_before', 'result', 'age_desc', 'relation', 'Class/ASD'])
df.iloc[0]

A1_Score                         1
A2_Score                         1
A3_Score                         1
A4_Score                         1
A5_Score                         0
A6_Score                         0
A7_Score                         1
A8_Score                         1
A9_Score                         0
A10_Score                        0
age                             26
gender                           f
ethnicity           White-European
jaundice                        no
austim                          no
country_of_res     'United States'
used_app_before                 no
result                           6
age_desc             '18 and more'
relation                      Self
Class/ASD                       NO
Name: 0, dtype: object

### Q19 [3 points] Create a target column which is 1 if Class/ASD is YES, and 0 otherwise.

Print out the baseline accuracy on this dataset.

### Q20 [6 points] Minimum number of questions

Calculate the average cross-validation accuracy for predicting the target using 10 different classifiers. Each classifier is a Nearest Neighbors classifier with K=15 nearest neighbors (all other settings can be set to their defaults), but with different features.
* Classifier 1 uses A1_Score
* Classifier 2 uses A1_Score and A2_Score
* Classifier 3 uses A1_Score, A2_Score, and A3_Score
* ...
* Classifier 10 uses A1_Score, A2_Score, ..., A10_Score

None of the classifiers use any of the other available columns such as age/gender/etc.

Find the maximum cross-validation accuracy among these 10 classifiers.
Then find out which classifiers achieve a cross-validation accuracy that is at least 95% of this maximum.
Among these, print out which classifier uses the fewest features.